In [1]:
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Input, Conv2D, MaxPooling2D, BatchNormalization, Lambda
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.models import load_model
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        if img is not None:
            images.append(img)
    return images

In [3]:
def get_num_classes():
    persons = os.listdir('data')
    num_classes = len(persons)
    return num_classes

In [4]:
def create_training_data(target_size):
    images = []
    labels = []
    persons = os.listdir('data')
    num_classes = len(persons)

    for person_index, person in enumerate(persons):
        folder_path = os.path.join('data', person)
        person_images = load_images_from_folder(folder_path)
        resized_images = [cv2.resize(img, target_size) for img in person_images]
        images.extend(resized_images)
        labels.extend([person_index] * len(resized_images))

    images = np.array(images)
    labels = np.array(labels)

    return images, labels, num_classes

In [5]:
def create_model():
    num_classes=get_num_classes()
    
    input_shape = (150, 150, 3)
    
    model = Sequential()
    model.add(Conv2D(128, (3, 3), activation='relu', input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    return model

model = create_model()

In [6]:
target_size = (150, 150)
images, labels, num_classes = create_training_data(target_size)

x_train, x_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42, stratify=labels)
y_train = np_utils.to_categorical(y_train, num_classes)
y_val = np_utils.to_categorical(y_val, num_classes)
x_train = x_train / 255.0
x_val = x_val / 255.0

learning_rate = 0.001
batch_size = 32
epochs = 50

optimizer = Adam(learning_rate=learning_rate)

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'], run_eagerly=True)

datagen_train = ImageDataGenerator(rotation_range=20, width_shift_range=0.2, height_shift_range=0.2,
                                   horizontal_flip=True)

datagen_train.fit(x_train)

In [7]:
checkpoint = ModelCheckpoint('model/retrained.h5', monitor='val_accuracy', verbose=1,
                             save_best_only=True, mode='max')

history = model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs, 
    shuffle=True,
    validation_data=(x_val, y_val), 
    callbacks=[checkpoint]
)

Epoch 1/50
10/10 [==============================] - ETA: 0s - loss: 59.3706 - accuracy: 0.1336
Epoch 1: val_accuracy improved from -inf to 0.16883, saving model to model\retrained.h5
10/10 [==============================] - 15s 2s/step - loss: 59.3706 - accuracy: 0.1336 - val_loss: 2.6018 - val_accuracy: 0.1688
Epoch 2/50
10/10 [==============================] - ETA: 0s - loss: 10.4061 - accuracy: 0.2443
Epoch 2: val_accuracy did not improve from 0.16883
10/10 [==============================] - 13s 1s/step - loss: 10.4061 - accuracy: 0.2443 - val_loss: 2.4226 - val_accuracy: 0.1039
Epoch 3/50
10/10 [==============================] - ETA: 0s - loss: 2.8654 - accuracy: 0.2801
Epoch 3: val_accuracy did not improve from 0.16883
10/10 [==============================] - 13s 1s/step - loss: 2.8654 - accuracy: 0.2801 - val_loss: 2.3020 - val_accuracy: 0.1039
Epoch 4/50
10/10 [==============================] - ETA: 0s - loss: 2.1583 - accuracy: 0.2606
Epoch 4: val_accuracy did not improve from 

10/10 [==============================] - ETA: 0s - loss: 0.5499 - accuracy: 0.7850
Epoch 30: val_accuracy did not improve from 0.23377
10/10 [==============================] - 14s 1s/step - loss: 0.5499 - accuracy: 0.7850 - val_loss: 2.2081 - val_accuracy: 0.1818
Epoch 31/50
10/10 [==============================] - ETA: 0s - loss: 0.5713 - accuracy: 0.7850
Epoch 31: val_accuracy did not improve from 0.23377
10/10 [==============================] - 15s 2s/step - loss: 0.5713 - accuracy: 0.7850 - val_loss: 2.1736 - val_accuracy: 0.2208
Epoch 32/50
10/10 [==============================] - ETA: 0s - loss: 0.5626 - accuracy: 0.7915
Epoch 32: val_accuracy improved from 0.23377 to 0.25974, saving model to model\retrained.h5
10/10 [==============================] - 17s 2s/step - loss: 0.5626 - accuracy: 0.7915 - val_loss: 2.2221 - val_accuracy: 0.2597
Epoch 33/50
10/10 [==============================] - ETA: 0s - loss: 0.5612 - accuracy: 0.7980
Epoch 33: val_accuracy did not improve from 0.259

In [8]:
test_images, test_labels, _ = create_training_data(target_size)
test_images = test_images / 255.0
test_labels = np_utils.to_categorical(test_labels, num_classes)
model.load_weights('model/retrained.h5')
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
test_loss, test_accuracy = model.evaluate(test_images, test_labels, verbose=0)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)

Test Loss: 0.7541497349739075
Test Accuracy: 0.8463541865348816


In [9]:
def test(name_image):
    image_path = 'test/' + name_image
    image = cv2.imread(image_path)
    image = cv2.resize(image, (150, 150))
    image = np.expand_dims(image, axis=0)
    image = image / 255.0
    predictions = model.predict(image)
    persons = os.listdir('data')
    predicted_person_index = np.argmax(predictions[0])
    predicted_person = persons[predicted_person_index]
    print(predicted_person)

In [19]:
test('test_1.jpg')
test('test_2.jpg')
test('test_3.jpg')

1/1 [==============================] - 0s 38ms/step
Lev Tolstoy
1/1 [==============================] - 0s 49ms/step
Winston Churchill
1/1 [==============================] - 0s 49ms/step
Quentin Tarantino
